# Instamatic

Instamatic is a tool for automated electron diffraction data collection. It has interfaces for interfacing with the TEM (JEOL/TFS) and several cameras (Gatan/ASI Timepix/TVIPS).

https://github.com/stefsmeets/instamatic

This notebook shows how to set up a grid montage

First, we must import and initialize the controller object, which defines the interface to the TEM and the camera. For this example, the microscope interaction is simulated.

In [ ]:
from instamatic import TEMController
from instamatic.formats import write_tiff

ctrl = TEMController.initialize()

Next, we will set the image conditions. We want to collect a montage over 5 by 5 points using the `lowmag` mode at a 100x magnification

In [ ]:
ctrl.mode = "lowmag"
ctrl.magnification.value = 100
grid_x, grid_y = 3, 3

## Setting up the grid montage

The grid montage can be set up as follows:

In [ ]:
from instamatic.montage import *

gm = ctrl.grid_montage()
pos = gm.setup(5, 5)

gm.plot()

Internally, the acquisition is done using the `ctrl.acquire_at_points` function. Acquisition is started using `gm.start`. Afterwards, the data are saved to a new experiment directory.

In [ ]:
gm.start()

Finally, we use `gm.to_montage` to give us a `Montage` object, which can be used to stitch the images together.

In [ ]:
m = gm.to_montage()

First, we can check what the data actually look like. To do so, we can simply stitch and plot the data using a binning=4 to conserve a bit of memory. This naively plots the data at the expected positions. Although this may not always give good results, it's enough to get a feeling for the data.

In [ ]:
# calculate the montage coordinates
m.calculate_montage_coords()

# stitch image, use binning 4 for speed-up and memory conservation
stitched = m.stitch(binning=4)

# plot the stitched image
m.plot()

To get better stitching, we can do two things:
 1. Better estimate the difference vectors between each tile using cross correlation
 2. Optimize the coordinates of the difference vectors using least-squares minimization

In [ ]:
# Use cross correlation to get difference vectors
m.calculate_difference_vectors(threshold='auto', verbose=False)

# plot the fft_scores also showing the threshold used
m.plot_fft_scores()

# get coords optimized using cross correlation
m.optimize_montage_coords()

# stitch image, use binning 4 for speed-up and memory conservation
m.stitch(binning=4)

# plot the stitched image
m.plot()